# IMDb API

### links 

http://www.omdbapi.com/#usage

https://www.imdb.com

### Required imports

In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import requests
from bs4 import BeautifulSoup
import webbrowser
import cv2
import numpy as np
import json

### Class creation  

In [ ]:
class SearchKeyword:
    
    """ This class searches in the API with the given keyword """
    
    def __init__(self, name, types=''):
        self.name = name
        self.types = types

    def search(self):
        
        """ The functions gets an instance and returns the search result in JSON format """
        
        res = requests.get(
            f'http://www.omdbapi.com/?s={self.name}&type={self.types}&apikey=87bc2753'
        )
        out = res.json()
        return out


class SearchTitle:
    
    """ This class searches in the API with the given title """
        
    def __init__(self, full_name):
        self.full_name = full_name

    def search(self):
        
        """ The functions gets an instance and returns the search result in JSON format """
        
        res = requests.get(
            f'http://www.omdbapi.com/?t={self.full_name}&apikey=87bc2753'
        )
        out = res.json()
        return out


In [ ]:
class Windows:
    
    """ This class creates all the tkinter windows following after the first welcome window """
    @staticmethod
    def search_full_name(res):
        
        """ This function creates all the windows following after user's choice to search with a full name.
            res - the output of class SearchTitle's search function
        """
        
        link = res
        def trailers():
            
            """ This function opens the selected film's or serie's trailer with webbrouser """
            
            url = f"https://www.imdb.com/find?q={link['Title']}&ref_=nv_sr_sm"
            headers = {
                "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'
            }
            res = requests.get(url, headers=headers)
            soup = BeautifulSoup(res.text, 'html.parser')
            item = soup.find(class_="result_text").find('a')['href']
            url2 = 'https://www.imdb.com' + item
            webbrowser.open(url2)
        
        
        def img():
            
            """ This function opens the selected film's or serie's poster with openCV """
            
            url = link['Poster']
            resp = requests.get(url, stream=True).raw
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)

            cv2.imshow('image', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        def exit_system():
            window.destroy()

        rows = []
        for i, j in res.items():
            rows.append([i, j])
            
        columns = ["1", "2"]
        
        window = tk.Tk()
        window.geometry('1400x550')
        window.title(f"Search")
            
        tree = ttk.Treeview(window, columns=columns, show='headings')
        
        tk.Label(window, bg='white').grid(row=1, column=0, stick='we', columnspan=3)
        image = tk.Button(window, text='Image', fg='white', bg='grey', font=('Arial', 10), command=img)
        image.grid(row=2, column=0, stick='we', columnspan=3)
        
        tk.Label(window, bg='white').grid(row=3, column=0, stick='we', columnspan=3)
        image = tk.Button(window, text='Trailer', fg='white', bg='grey', font=('Arial', 10), command=trailers)
        image.grid(row=4, column=0, stick='we', columnspan=3)
        
        tk.Label(window, bg='white').grid(row=5, column=0, stick='we', columnspan=3)
        exit_btn = tk.Button(window, text='Exit', fg='white', bg='grey', font=('Arial', 10), command=exit_system)
        exit_btn.grid(row=6, column=0, stick='we', columnspan=3)
            
        for column in columns:
            tree.heading(column, text=column)

        for row in rows:
            tree.insert('', tk.END, values=row)


        def item_selected(event):
            for selected_item in tree.selection():
                item = tree.item(selected_item)
                record = item['values']

        tree.bind('<<TreeviewSelect>>', item_selected)
        tree.grid(row=0, column=0, sticky='nsew')
    
        scrollbar = ttk.Scrollbar(window, orient=tk.VERTICAL, command=tree.yview)
        tree.configure(yscroll=scrollbar.set)
        scrollbar.grid(row=0, column=1, sticky='ns')
            
        window.grid_columnconfigure(0, weight=1)
        window.grid_rowconfigure(0, weight=1)
            
        window.mainloop()
    
    @staticmethod
    def search_keyword(res):
        
        """ This function creates the window following after user's choice to search with a keyword.
            res - the output of class SearchKeyword's search function
        """
        
        rows = []
        for i in res['Search']:
            row = []
            for j in i.values():
                row.append(j)
            rows.append(tuple(row))
                
        columns = ["Title", "Year", "imdbID", "Type", "Poster"]
        
        window = tk.Tk()
        window.geometry('1400x400')
        window.title(f"Search")
            
        tree = ttk.Treeview(window, columns=columns, show='headings')
            
        for column in columns:
            tree.heading(column, text=column)

        for row in rows:
            tree.insert('', tk.END, values=row)


        def item_selected(event):
            for selected_item in tree.selection():
                item = tree.item(selected_item)
                record = item['values']

        tree.bind('<<TreeviewSelect>>', item_selected)
        tree.grid(row=0, column=0, sticky='nsew')
    
        scrollbar = ttk.Scrollbar(window, orient=tk.VERTICAL, command=tree.yview)
        tree.configure(yscroll=scrollbar.set)
        scrollbar.grid(row=0, column=1, sticky='ns')
            
        scrollbar_h = ttk.Scrollbar(window, orient=tk.HORIZONTAL, command=tree.xview)
        tree.configure(xscroll=scrollbar.set)
        scrollbar_h.grid(row=1, column=0, sticky='ew')
            
        window.grid_columnconfigure(0, weight=1)
        window.grid_rowconfigure(0, weight=1)
            
        window.mainloop() 
        
    @staticmethod
    def choose(tps, types):
        
        """ This function creates the window where users can choose the search option
            and input the keyword or full name accordingly """
        
        def exit_system():
            window.destroy()
        def submit2():
            if box.get() == 'Keyword':
                keyword = entry.get()
                if types == "movie":
                    find = SearchKeyword(keyword, types="movie")
                else:
                    find = SearchKeyword(keyword, types="series")
                    
                if find.search()["Response"] == "False":
                    messagebox.showinfo('Attention',"Not found!")
                else:
                    Windows.search_keyword(find.search())
            else:
                full_name = entry.get()
                find = SearchTitle(full_name)
                if find.search()["Response"] == "False":
                    messagebox.showinfo('Attention',"Not found!")
                else:
                    Windows.search_full_name(find.search())  

                    
        
        window = tk.Tk()
        window.title(f"{tps}")
        window.geometry('350x350+550+250')
        window['bg'] = 'white'
        window.resizable(False, False)

        for i in [1, 3, 5, 6, 8, 10]:
            tk.Label(window, bg='white').grid(row=i, column=0, stick='we', columnspan=2)
        tk.Label(window, text='Search by...', font=('Arial', 20, 'bold'), bg='white').grid(row=0, column=0,
                                                                                   stick='we',columnspan=2)

        tk.Label(window, text='type', font=('Arial', 15), bg='white').grid(row=2, column=0, stick='w')
        box = ttk.Combobox(window, values=['Keyword', 'Full name'], font=('Arial', 15))
        box.current(0)
        box.grid(row=2, column=1)
        
        tk.Label(window, text='Name', font=('Arial', 15), bg='white').grid(row=7, column=0, stick='w')
        entry = tk.Entry(window, bg='white', fg='black', font=('Arial', 15), justify=tk.RIGHT)
        entry.grid(row=7, column=1, stick='we')
        
        submit2 = tk.Button(window, text='Submit', fg='white', bg='grey', font=('Arial', 10), command=submit2)
        submit2.grid(row=9, column=1, stick='we', columnspan=3)

        exit_btn = tk.Button(window, text='Exit', fg='white', bg='grey', font=('Arial', 10), command=exit_system)
        exit_btn.grid(row=11, column=1, stick='we', columnspan=3)


        window.mainloop()

    
    
        

### Interface 

In [ ]:
class WelcomePage:
    
    """ This class is responsible for creating the first welcome page """
    
    @staticmethod
    def welcome():
        
        """This function creates the first wecome page and the program starts from this function"""

        def exit_system():
                window.destroy()

        def submit():    
            if (movie_check.get() == "Yes") and (series_check.get() == "Yes"):
                messagebox.showinfo('Attention','Please choose one of the options')
                movie_check.set('No')
                series_check.set('No')
            elif (movie_check.get() == "Yes") and (series_check.get() == "No"):
                Windows.choose('Movie', "movie")
            elif (movie_check.get() == "No") and (series_check.get() == "Yes"):
                Windows.choose('Series', 'series')
            else:
                messagebox.showinfo('Attention','Please choose option')


        window = tk.Tk()
        window.title("Welcome")
        window.geometry('400x300+500+250')
        window['bg'] = 'white'
        window.resizable(False, False)

        for i in [1, 3, 5, 7, 9]:
            tk.Label(window, bg='white').grid(row=i, column=0, stick='we', columnspan=2)
        tk.Label(window, text='Hello my name is Kinogid!.', font=('Arial', 15, 'bold'), bg='white').grid(row=0, column=0,
                                                                                                  stick='we',columnspan=2)

        tk.Label(window, text='What do you want to serach for?', font=('Arial', 12), bg='white').grid(row=2, column=0,
                                                                                                  stick='w',columnspan=2)

        movie_check = tk.StringVar()
        movie_check.set('No')
        movie = tk.Checkbutton(window, text='Movie', font=('Arial', 12), bg='white', variable=movie_check, onvalue="Yes",
                              offvalue="No").grid( row=4, column=0,stick='w',columnspan=2)

        series_check = tk.StringVar()
        series_check.set('No')
        series = tk.Checkbutton(window, text='Series', font=('Arial', 12), bg='white', variable=series_check, onvalue="Yes",
                              offvalue="No").grid( row=6, column=0,stick='w',columnspan=2)


        submit1 = tk.Button(window, text='Submit', fg='white', bg='grey', font=('Arial', 10), command=submit)
        submit1.grid(row=8, column=1, stick='we', columnspan=3)

        exit_btn = tk.Button(window, text='Exit', fg='white', bg='grey', font=('Arial', 10), command=exit_system)
        exit_btn.grid(row=10, column=1, stick='we', columnspan=3)


        window.mainloop()



In [ ]:
WelcomePage.welcome()